In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join
import torch
import torch as th
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm.auto import trange
from contextlib import redirect_stdout

import sys
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha")
from diffusion import IDDPM
from diffusion.data.builder import build_dataset, build_dataloader, set_data_root
from diffusion.model.builder import build_model
from diffusion.utils.misc import set_random_seed, read_config, init_random_seed, DebugUnderflowOverflow
from diffusers import AutoencoderKL, Transformer2DModel, PixArtAlphaPipeline, DPMSolverMultistepScheduler
from transformers import T5Tokenizer, T5EncoderModel

sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation")
from utils.pixart_sampling_utils import pipeline_inference_custom, \
    PixArtAlphaPipeline_custom
from utils.pixart_utils import state_dict_convert
from utils.text_encoder_control_lib import RandomEmbeddingEncoder_wPosEmb
from utils.image_utils import pil_images_to_grid
from utils.attention_map_store_utils import replace_attn_processor, AttnProcessor2_0_Store, PixArtAttentionVisualizer_Store
from utils.cv2_eval_utils import find_classify_object_masks
from utils.attention_analysis_lib import plot_attention_layer_head_heatmaps, plot_layer_head_score_summary
from utils.attention_analysis_lib import *
from utils.cv2_eval_utils import find_classify_object_masks
from utils.obj_mask_utils import *
from circuit_toolkit.plot_utils import saveallforms

/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Pleas

In [6]:
project_root = f"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/"
dataset_root = f"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets"

In [7]:
!ls {project_root}/training_datasets

objectRelDouble_pilot1_T5     objectRelSingle_pilot1_RndEmbPos
objectRelMixed_pilot1_T5      objectRelSingle_pilot1_T5
objectRelMulti_pilot1	      objectRelSingle_pilot3
objectRel_pilot		      objectRelSingle_pilot4
objectRel_pilot2	      objectRelSingle_pilot_RndEmbPos
objectRel_pilot_rndemb	      objectRelSingle_pilot_temp
objectRel_pilot_rndembposemb  objectRelSingle_randomized_pilot1
objectRel_pilot_T5	      pixart-pokemon
objectRelSingle_pilot1


In [8]:
"objectRelSingle_pilot1_T5"
"objectRelDouble_pilot1_T5"
"objectRelMixed_pilot1_T5"

'objectRelMixed_pilot1_T5'

In [ ]:
dataset_name = "objectRelMixed_pilot1_T5"
dataset_dir = join(dataset_root, dataset_name)

In [16]:
!ls {dataset_dir}/

caption_feature_wmask  images			       partition
captions	       img_vae_features_128resolution


In [ ]:
import numpy as np
text_feat_dir = join(dataset_dir, "caption_feature_wmask")
data = np.load(join(text_feat_dir, "1.npz"))
print(data["caption_feature"].shape, data["caption_feature"].dtype)

(1, 20, 4096) float32


In [15]:
np.linalg.norm(data["caption_feature"][0],axis=-1)

array([4.5143533, 6.321398 , 6.9485   , 2.593737 , 8.047104 , 7.0711546,
       6.4765143, 8.275869 , 6.6898527, 7.0959353, 7.9806247, 7.380548 ,
       7.9859247, 7.7035613, 7.8594246, 7.3325157, 8.319464 , 8.063627 ,
       8.624287 , 7.344226 ], dtype=float32)

In [ ]:
caption_dir = join(dataset_dir, "captions")
prompt_files = [f for f in os.listdir(caption_dir) if f.endswith(".txt")]
with open(join(caption_dir, prompt_files[0]), 'r') as f:
    prompt = f.read().strip()
print(prompt)

triangle is above and to the right of circle


### Token ize with T5

In [19]:
T5_path = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models/t5_ckpts/t5-v1_1-xxl"
tokenizer = T5Tokenizer.from_pretrained(T5_path, )#subfolder="tokenizer")
# encoder = T5EncoderModel.from_pretrained(T5_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [21]:
caption_dir = join(dataset_dir, "captions")
model_max_length = 20
# use T5 tokenizer 
corpus = []
input_ids_col = []
attention_mask_col = []
for i in trange(10000):
    text = open(join(caption_dir, f"{i}.txt")).read()
    text_tokens_and_mask = tokenizer(
        text,
        max_length=model_max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    input_ids_col.append(text_tokens_and_mask['input_ids'])
    attention_mask_col.append(text_tokens_and_mask['attention_mask'])
    corpus.append(text)

input_ids_tsr = th.cat(input_ids_col, dim=0)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
unique_input_ids, unique_indices = th.unique(input_ids_tsr, return_inverse=True)


In [23]:
# print out the meaning of the unique input ids 
for i, ids in enumerate(unique_input_ids):
    print(i, ids.item(), tokenizer.decode(ids))

0 0 <pad>
1 1 </s>
2 3 
3 8 the
4 9 a
5 11 and
6 12 to
7 13 of
8 19 is
9 42 or
10 45 from
11 46 an
12 95 up
13 120 ly
14 145 than
15 269 right
16 323 down
17 646 left
18 666 below
19 756 above
20 1131 red
21 1146 higher
22 1364 lower
23 1461 directly
24 1692 blue
25 2812 square
26 4548 upper
27 8196 circle
28 19938 triangle
29 26184 diagonal


In [37]:
assert (input_ids_tsr == unique_input_ids[unique_indices]).all()

In [ ]:

old_word_embedding_dict = th.load("/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/objectRel_pilot_rndemb/caption_feature_wmask/word_embedding_dict.pt")
old_unique_input_ids = old_word_embedding_dict["unique_input_ids"]
old_embedding_dict = old_word_embedding_dict["embedding_dict"]
old_input_ids2dict_ids = old_word_embedding_dict["input_ids2dict_ids"]
old_dict_ids2input_ids = old_word_embedding_dict["dict_ids2input_ids"]

tensor([    0,     1,     8,    11,    12,    13,    19,    45,    95,   120,
          145,   269,   323,   646,   666,   756,  1131,  1146,  1364,  1461,
         1692,  2812,  4548,  8196, 19938, 26184])

In [ ]:
old_input_ids2dict_ids

{0: 0,
 1: 1,
 8: 2,
 11: 3,
 12: 4,
 13: 5,
 19: 6,
 45: 7,
 95: 8,
 120: 9,
 145: 10,
 269: 11,
 323: 12,
 646: 13,
 666: 14,
 756: 15,
 1131: 16,
 1146: 17,
 1364: 18,
 1461: 19,
 1692: 20,
 2812: 21,
 4548: 22,
 8196: 23,
 19938: 24,
 26184: 25}

In [ ]:
os.makedirs(join(dataset_root, "random_emd_dictionary"), exist_ok=True)

In [40]:
# load in the old word embedding dict 
old_word_embedding_dict = th.load("/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/objectRel_pilot_rndemb/caption_feature_wmask/word_embedding_dict.pt")
old_unique_input_ids = old_word_embedding_dict["unique_input_ids"]
old_embedding_dict = old_word_embedding_dict["embedding_dict"]
old_input_ids2dict_ids = old_word_embedding_dict["input_ids2dict_ids"]
old_dict_ids2input_ids = old_word_embedding_dict["dict_ids2input_ids"]
# load in the new word embedding dict 
# new_word_embedding_dict = th.load(join(dataset_root, "word_embedding_dict_randemb_new.pt"))
# print out the meaning of the unique input ids 
for i, ids in enumerate(old_unique_input_ids):
    print(i, ids.item(), tokenizer.decode(ids))
    
# for the new embedding dict if the entry exists in the old embedding dict, then use the old embedding, otherwise use the new embedding
embed_dim = 4096
unique_input_ids, unique_indices = th.unique(input_ids_tsr, return_inverse=True)
input_ids2dict_ids = {idx.item(): id for id, idx in enumerate(unique_input_ids)}
dict_ids2input_ids = {id: idx.item() for id, idx in enumerate(unique_input_ids)}
new_embedding_dict = th.zeros(len(unique_input_ids), embed_dim, dtype=th.float32)
# new_embedding_dict = th.randn(len(unique_input_ids), embed_dim, 
#                           generator=th.Generator().manual_seed(42)) \
#                             / np.sqrt(embed_dim) * 7.5
# for the new embedding dict if the entry exists in the old embedding dict, then use the old embedding, otherwise use the new embedding
for i, ids in enumerate(unique_input_ids):
    ids = ids.item()
    if ids in old_unique_input_ids:
        # if it's in the old embedding dict, then use the old embedding
        new_embedding_dict[i] = old_embedding_dict[old_input_ids2dict_ids[ids]]
    else:
        new_embedding_dict[i] = th.randn(1, embed_dim, 
                          generator=th.Generator().manual_seed(i)) \
                            / np.sqrt(embed_dim) * 7.5


th.save({"unique_input_ids": unique_input_ids, 
         "embedding_dict": new_embedding_dict,
         "input_ids2dict_ids": input_ids2dict_ids, 
         "dict_ids2input_ids": dict_ids2input_ids}, 
         join(dataset_root, "random_emd_dictionary", "word_embedding_dict_randemb_new.pt"))

0 0 <pad>
1 1 </s>
2 8 the
3 11 and
4 12 to
5 13 of
6 19 is
7 45 from
8 95 up
9 120 ly
10 145 than
11 269 right
12 323 down
13 646 left
14 666 below
15 756 above
16 1131 red
17 1146 higher
18 1364 lower
19 1461 directly
20 1692 blue
21 2812 square
22 4548 upper
23 8196 circle
24 19938 triangle
25 26184 diagonal


/tmp/ipykernel_1286049/3673783431.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  old_word_embedding_dict = th.load("/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/bin

### Final tests

In [46]:
new_dict_path = join(dataset_root, "random_emd_dictionary", "word_embedding_dict_randemb_new.pt")
new_word_embedding_dict = th.load(new_dict_path)
old_dict_path = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/objectRel_pilot_rndemb/caption_feature_wmask/word_embedding_dict.pt"
old_word_embedding_dict = th.load(old_dict_path)

/tmp/ipykernel_1286049/1554418953.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_word_embedding_dict = th.load(new_dict_path)
/tmp/ipykernel_1286049/1554418953.py:4

In [45]:
import sys
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation/")
from utils.text_encoder_control_lib import RandomEmbeddingEncoder, RandomEmbeddingEncoder_wPosEmb

In [47]:
old_encoder = RandomEmbeddingEncoder(embedding_dict=old_word_embedding_dict["embedding_dict"], 
                                    input_ids2dict_ids=old_word_embedding_dict["input_ids2dict_ids"], 
                                    dict_ids2input_ids=old_word_embedding_dict["dict_ids2input_ids"])
new_encoder = RandomEmbeddingEncoder(embedding_dict=new_word_embedding_dict["embedding_dict"], 
                                    input_ids2dict_ids=new_word_embedding_dict["input_ids2dict_ids"], 
                                    dict_ids2input_ids=new_word_embedding_dict["dict_ids2input_ids"])

In [57]:

for caption_i in range(100):
    try:
        caption_emb_old, _ = old_encoder(input_ids_tsr[caption_i])
        caption_emb_new, _ = new_encoder(input_ids_tsr[caption_i])
        assert torch.allclose(caption_emb_old, caption_emb_new)
    except KeyError as e:
        print(f"Caption {caption_i} is not in the old encoder")
        print(e)
        break

Caption 12 is not in the old encoder
42


In [60]:
old_encoder = RandomEmbeddingEncoder_wPosEmb(embedding_dict=old_word_embedding_dict["embedding_dict"], 
                                    input_ids2dict_ids=old_word_embedding_dict["input_ids2dict_ids"], 
                                    dict_ids2input_ids=old_word_embedding_dict["dict_ids2input_ids"]).to("cuda")
new_encoder = RandomEmbeddingEncoder_wPosEmb(embedding_dict=new_word_embedding_dict["embedding_dict"], 
                                    input_ids2dict_ids=new_word_embedding_dict["input_ids2dict_ids"], 
                                    dict_ids2input_ids=new_word_embedding_dict["dict_ids2input_ids"]).to("cuda")

In [63]:
input_ids_tsr.shape

torch.Size([10000, 20])

In [65]:
for caption_i in range(10000):
    try:
        caption_emb_old, _ = old_encoder(input_ids_tsr[caption_i])
        caption_emb_new, _ = new_encoder(input_ids_tsr[caption_i])
        assert torch.allclose(caption_emb_old, caption_emb_new)
    except KeyError as e:
        # print(f"Caption {caption_i} is not in the old encoder")
        print(f"missing key: {e}")
        continue

missing key: 42
missing key: 42
missing key: 3
missing key: 3
missing key: 3
missing key: 46
missing key: 3
missing key: 46
missing key: 3
missing key: 3
missing key: 46
missing key: 46
missing key: 46
missing key: 42
missing key: 42
missing key: 3
missing key: 42
missing key: 46
missing key: 3
missing key: 46
missing key: 46
missing key: 46
missing key: 3
missing key: 42
missing key: 3
missing key: 42
missing key: 46
missing key: 46
missing key: 3
missing key: 46
missing key: 46
missing key: 3
missing key: 42
missing key: 46
missing key: 3
missing key: 3
missing key: 3
missing key: 46
missing key: 42
missing key: 3
missing key: 46
missing key: 3
missing key: 3
missing key: 3
missing key: 42
missing key: 3
missing key: 3
missing key: 46
missing key: 46
missing key: 46
missing key: 42
missing key: 3
missing key: 46
missing key: 46
missing key: 3
missing key: 42
missing key: 3
missing key: 3
missing key: 46
missing key: 3
missing key: 46
missing key: 3
missing key: 42
missing key: 46
mis

In [41]:
join(dataset_root, "random_emd_dictionary", "word_embedding_dict_randemb_new.pt")

'/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/random_emd_dictionary/word_embedding_dict_randemb_new.pt'

In [66]:
!cp /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/objectRel_pilot_rndemb/caption_feature_wmask/word_embedding_dict.pt /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/random_emd_dictionary

In [36]:
len(new_embedding_dict)

30

In [35]:
len(torch.unique(new_embedding_dict.norm(dim=-1)))

30

In [30]:
ids

tensor(0)

In [ ]:
embed_dim = 4096
# find unique input ids 
unique_input_ids, unique_indices = th.unique(input_ids_tsr, return_inverse=True)
input_ids2dict_ids = {idx.item(): id for id, idx in enumerate(unique_input_ids)}
dict_ids2input_ids = {id: idx.item() for id, idx in enumerate(unique_input_ids)}
# create random word embeddings
embedding_dict = th.randn(len(unique_input_ids), embed_dim, 
                          generator=th.Generator().manual_seed(42)) \
                            / np.sqrt(embed_dim) * 7.5
# check if the unique input ids are the same as the input ids 
print(input_ids_tsr[0])
print(unique_input_ids[unique_indices[0]])
assert (input_ids_tsr == unique_input_ids[unique_indices]).all()
th.save({"unique_input_ids": unique_input_ids, 
         "embedding_dict": embedding_dict,
         "input_ids2dict_ids": input_ids2dict_ids, 
         "dict_ids2input_ids": dict_ids2input_ids}, 
         join(dataset_root, "word_embedding_dict_randemb_new.pt"))

In [ ]:
# print out the meaning of the unique input ids 
for i, ids in enumerate(unique_input_ids):
    print(i, ids.item(), tokenizer.decode(ids))

In [ ]:

caption_token = tokenizer(prompt, max_length=max_length, padding="max_length", 
                          truncation=True, return_tensors="pt").to(device)